In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d aravindanr22052001/emotiondetection-happy-or-sad

Dataset URL: https://www.kaggle.com/datasets/aravindanr22052001/emotiondetection-happy-or-sad
License(s): CC-BY-NC-SA-4.0
 77% 50.0M/64.8M [00:00<00:00, 111MB/s]
100% 64.8M/64.8M [00:00<00:00, 120MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/emotiondetection-happy-or-sad.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense , Conv2D,MaxPooling2D ,Flatten,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split
import cv2

In [12]:
! pip install split-folders

In [15]:
import splitfolders

input_file ='/content/data/'
output_file = '/content/happy_train'

In [16]:
splitfolders.ratio(input_file,output=output_file,seed=38,ratio=(.75 , .25),group_prefix=None)

Copying files: 168 files [00:00, 308.66 files/s]


In [17]:
train_ds= keras.utils.image_dataset_from_directory(
    directory ='/content/happy_train/train' ,
     labels="inferred",
    label_mode="int",
     batch_size=32,
    image_size=(256, 256)
)

val_ds= keras.utils.image_dataset_from_directory(
    directory ='/content/happy_train/val' ,
     labels="inferred",
    label_mode="int",
     batch_size=32,
    image_size=(256, 256)
)

Found 125 files belonging to 2 classes.
Found 43 files belonging to 2 classes.


In [18]:
# Normalization (so that the pixel values are between 0 and 1)
def process(image,label):
  image = tf.cast(image/255. ,tf.float32)
  return image,label

train_ds=train_ds.map(process)
val_ds =val_ds.map(process)

In [29]:
model=Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 128)      

In [26]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
history = model.fit(train_ds,epochs=25,validation_data=val_ds)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_3529]